# Load Dataset

In [ ]:
import argparse
import pandas as pd
import numpy as np

#parser = argparse.ArgumentParser(description="Set board size for Hex dataset.")
#parser.add_argument("--board_size", type=int, default=3, help="Size of the Hex board (default: 7)")

#args = parser.parse_args()

board_size = 4

n_turns_before_win = 2
n_games = 10000

path = f'hex_datasets/hex_{board_size}x{board_size}_{n_games}_games_{n_turns_before_win}_turns_before_win.csv'

df = pd.read_csv(path)

train_split = 0.8

train_end_index = int(len(df)*train_split)

print(train_end_index)

df_train = df[0:train_end_index]
df_test = df[train_end_index:]
print(len(df_train), len(df_test))

FileNotFoundError: [Errno 2] No such file or directory: 'hex_datasets/hex_2x2_10000_games_2_turns_before_win.csv'

In [186]:
from GraphTsetlinMachine.graphs import Graphs

HYPERVECTOR_SIZE = 120
HYPERVECTOR_BITS = 2

x_positon_symbols = [f'x:{x}' for x in range(board_size + 2)]
y_positon_symbols = [f'y:{y}' for y in range(board_size + 2)]

graph_node_args = {
    'symbols' : ['PlayerX', 'PlayerXEdge', 'Empty','PlayerO', 'PlayerOEdge'] + x_positon_symbols + y_positon_symbols,
    'hypervector_size': HYPERVECTOR_SIZE,
    'hypervector_bits': HYPERVECTOR_BITS
}


assert board_size == int(np.sqrt(len(df_train.iloc[0, :-1])))

num_nodes = board_size*board_size + (2*(board_size+1)) + (2 * board_size)     #Adds edges as colored nodes

print(num_nodes)


79


## Get all Nodes for the board dimension and add initialize graphs

In [187]:

all_node_names = [f'{i}:{j}' for i in range(board_size+2) for j in range(board_size+2)]
print(all_node_names)

all_node_names = all_node_names[1:-1]     #Get all nodes including edges represented as nodes
print(all_node_names)


graphs_train = Graphs(
    train_end_index,
    **graph_node_args
)


graphs_test = Graphs(
    len(df_test),
    **graph_node_args,
    init_with=graphs_train
)

#Initalize graphs
for graph_id in range(len(df_train)):
    graphs_train.set_number_of_graph_nodes(graph_id, number_of_graph_nodes=np.uint32(len(all_node_names)))

for graph_id in range(len(df_test)):
    graphs_test.set_number_of_graph_nodes(graph_id, number_of_graph_nodes=np.uint32(len(all_node_names)))

    

['0:0', '0:1', '0:2', '0:3', '0:4', '0:5', '0:6', '0:7', '0:8', '1:0', '1:1', '1:2', '1:3', '1:4', '1:5', '1:6', '1:7', '1:8', '2:0', '2:1', '2:2', '2:3', '2:4', '2:5', '2:6', '2:7', '2:8', '3:0', '3:1', '3:2', '3:3', '3:4', '3:5', '3:6', '3:7', '3:8', '4:0', '4:1', '4:2', '4:3', '4:4', '4:5', '4:6', '4:7', '4:8', '5:0', '5:1', '5:2', '5:3', '5:4', '5:5', '5:6', '5:7', '5:8', '6:0', '6:1', '6:2', '6:3', '6:4', '6:5', '6:6', '6:7', '6:8', '7:0', '7:1', '7:2', '7:3', '7:4', '7:5', '7:6', '7:7', '7:8', '8:0', '8:1', '8:2', '8:3', '8:4', '8:5', '8:6', '8:7', '8:8']
['0:1', '0:2', '0:3', '0:4', '0:5', '0:6', '0:7', '0:8', '1:0', '1:1', '1:2', '1:3', '1:4', '1:5', '1:6', '1:7', '1:8', '2:0', '2:1', '2:2', '2:3', '2:4', '2:5', '2:6', '2:7', '2:8', '3:0', '3:1', '3:2', '3:3', '3:4', '3:5', '3:6', '3:7', '3:8', '4:0', '4:1', '4:2', '4:3', '4:4', '4:5', '4:6', '4:7', '4:8', '5:0', '5:1', '5:2', '5:3', '5:4', '5:5', '5:6', '5:7', '5:8', '6:0', '6:1', '6:2', '6:3', '6:4', '6:5', '6:6', '6:7', '6:8

# Initalize edges

In [188]:
graphs_train.prepare_node_configuration()
graphs_test.prepare_node_configuration()
from copy import copy

# Add corner edge nodes
remianing_nodes = copy(all_node_names)
corner_nodes = ['0:1', '1:0', f'{board_size}:{board_size+1}', f'{board_size+1}:{board_size}', f'{0}:{board_size+1}', f'{board_size+1}:{0}']
print('Corner Nodes:', corner_nodes)
for node_name in corner_nodes:
    remianing_nodes.remove(node_name)

#Add remaining edge nodes
remaining_edge_nodes = [node for node in remianing_nodes if ('0' in node.split(':') or f'{board_size+1}' in node.split(':'))]
print('Remaining Edge Nodes:', remaining_edge_nodes)
for node_name in remaining_edge_nodes:
    remianing_nodes.remove(node_name)

print('Number of remaining Board Nodes:', len(remianing_nodes))



#Initalise nodes in train graph
for graph_id in range(len(df_train)):
    for node_name in corner_nodes:
        graphs_train.add_graph_node(graph_id, node_name=node_name, number_of_graph_node_edges=1)
    
    for node_name in remaining_edge_nodes:
        graphs_train.add_graph_node(graph_id, node_name=node_name, number_of_graph_node_edges=2)
    
    for node_name in remianing_nodes:
        graphs_train.add_graph_node(graph_id, node_name=node_name, number_of_graph_node_edges=6)
    

for graph_id in range(len(df_test)):
    for node_name in corner_nodes:
        graphs_test.add_graph_node(graph_id, node_name=node_name, number_of_graph_node_edges=1)
    
    for node_name in remaining_edge_nodes:
        graphs_test.add_graph_node(graph_id, node_name=node_name, number_of_graph_node_edges=2)
    
    for node_name in remianing_nodes:
        graphs_test.add_graph_node(graph_id, node_name=node_name, number_of_graph_node_edges=6)
    
    

    


Corner Nodes: ['0:1', '1:0', '7:8', '8:7', '0:8', '8:0']
Remaining Edge Nodes: ['0:2', '0:3', '0:4', '0:5', '0:6', '0:7', '1:8', '2:0', '2:8', '3:0', '3:8', '4:0', '4:8', '5:0', '5:8', '6:0', '6:8', '7:0', '8:1', '8:2', '8:3', '8:4', '8:5', '8:6']
Number of remaining Board Nodes: 49


# Add Edges

In [189]:
graphs_train.prepare_edge_configuration()
graphs_test.prepare_edge_configuration()

directions = {
    (1, -1) : 'UpRight',
    (0, -1) : 'UpLeft',
    (1, 0) : 'Right',
    (-1, 0) : 'Left',
    (0, 1) : 'DownRight',
    (-1, 1) : 'DownLeft'
}


def add_offset_to_node(node_name, offset):
    #print(node_name, offset)
    i, j = map(int, node_name.split(':'))
    i += offset[0]
    j += offset[1]
    return ':'.join([str(i), str(j)])

all_offsets = directions.keys()
    


#Add remaining edge nodes
edges = []

#Add edges for all edge nodes
for node_name in all_node_names:
    for offset in all_offsets:
        neighbor = add_offset_to_node(node_name, offset)
        #print(neighbor)
        if node_name in corner_nodes or node_name in remaining_edge_nodes:      #For edge nodes, only add game tiles as edges.
            if neighbor in remianing_nodes:
                edges.append({
                    'from': node_name,
                    'to': neighbor,
                    'edge_type': directions[offset]
                })
        else:
            edges.append({
                    'from': node_name,
                    'to': neighbor,
                    'edge_type': directions[offset]
                })





for graph_id in range(len(df_train)):
    for edge in edges:
        graphs_train.add_graph_node_edge(graph_id, edge['from'], edge['to'], edge['edge_type'])

for graph_id in range(len(df_test)):
    for edge in edges:
        graphs_test.add_graph_node_edge(graph_id, edge['from'], edge['to'], edge['edge_type'])


        

# Add Graph Node Property and Extract winning player

In [190]:
Y_train = np.empty(len(df_train), dtype=np.uint32)

for graph_id, row in df_train.iterrows():
    for node_name in all_node_names:
        if node_name.split(":")[1] in ['0', f'{board_size+1}']:
            graphs_train.add_graph_node_property(graph_id, node_name=node_name, symbol='PlayerXEdge')
        elif node_name.split(":")[0] in ['0', f'{board_size+1}']:
            graphs_train.add_graph_node_property(graph_id, node_name=node_name, symbol='PlayerOEdge')
        else:
            assert node_name in remianing_nodes
            cell_value = row[f'cell{add_offset_to_node(node_name, (-1, -1)).replace(':', '_')}']
            if cell_value == -1:
                symbol = 'PlayerO'
            elif cell_value == 0:
                symbol = 'Empty'
            elif cell_value == 1:
                symbol = 'PlayerX'

            graphs_train.add_graph_node_property(graph_id, node_name=node_name, symbol=symbol)
        graphs_train.add_graph_node_property(graph_id, node_name=node_name, symbol=f'x:{node_name.split(':')[0]}')   #add x position symbol
        graphs_train.add_graph_node_property(graph_id, node_name=node_name, symbol=f'y:{node_name.split(':')[1]}')   #add y position symbol
    # Get winning player
    Y_train[graph_id] = row['winner'] if row['winner'] == 1 else 2

graphs_train.encode()


Y_test = np.empty(len(df_test), dtype=np.uint32)

for graph_id, row in df_test.iterrows():
    graph_id = graph_id - len(df_train)
    for node_name in all_node_names:
        if node_name.split(":")[1] in ['0', f'{board_size+1}']:
            graphs_test.add_graph_node_property(graph_id, node_name=node_name, symbol='PlayerXEdge')
        elif node_name.split(":")[0] in ['0', f'{board_size+1}']:
            graphs_test.add_graph_node_property(graph_id, node_name=node_name, symbol='PlayerOEdge')
        else:
            assert node_name in remianing_nodes
            cell_value = row[f'cell{add_offset_to_node(node_name, (-1, -1)).replace(':', '_')}']
            if cell_value == -1:
                symbol = 'PlayerO'
            elif cell_value == 0:
                symbol = 'Empty'
            elif cell_value == 1:
                symbol = 'PlayerX'
            graphs_test.add_graph_node_property(graph_id, node_name=node_name, symbol=symbol)
        graphs_test.add_graph_node_property(graph_id, node_name=node_name, symbol=f'x:{node_name.split(':')[0]}')   #add x position symbol
        graphs_test.add_graph_node_property(graph_id, node_name=node_name, symbol=f'y:{node_name.split(':')[1]}')   #add y position symbol
    # Get winning player
    Y_test[graph_id] = row['winner'] if row['winner'] == 1 else 2

graphs_test.encode()



In [191]:
##Logging function
import json
import os
run_number = -1
def log_if_best_result(accuracy_eval, **kwargs):
    existing_result = False
    log = False
    
        
    
    if run_number != -1:    #Default value
        result_folder = os.path.join('results/', f'{board_size}x{board_size}_board_{n_turns_before_win}_turns_before_win')
        file_name = f'run_{run_number}.json'
            
    else:
        result_folder = 'results/'
        file_name = f'{board_size}x{board_size}_board_{n_turns_before_win}_turns_before_win.json'
        
    if not os.path.exists(result_folder):
        os.makedirs(result_folder)

    path = os.path.join(result_folder, file_name)
    
    log_object = {'eval_accuracy' : accuracy_eval, 'n_train_games': n_games, **kwargs}
    if file_name not in os.listdir(result_folder):
        log = True
    else:
        existing_result = True
        with open(path, 'r') as f:
            json_object = json.load(f)
            best_accuracy = json_object['eval_accuracy']
            if accuracy_eval > best_accuracy:
                log = True
                
            elif accuracy_eval == 100:
                if log_object['TM_args']['number_of_clauses'] < json_object['TM_args']['number_of_clauses']:
                    log = True
    if log:
        print(f'New best result for board size {board_size}, {n_turns_before_win} turns before win\nAccuracy : {accuracy_eval} with {log_object['TM_args']['number_of_clauses']} clauses.')
        if existing_result:
            print(f'Previouis best accuracy was: {best_accuracy} with {json_object['TM_args']['number_of_clauses']} clauses.')
        print('Logging paramters')
        with open(path, 'w') as f:
            json.dump(log_object, f, indent=4)

# Smoke test

In [192]:
from GraphTsetlinMachine.tm import MultiClassGraphTsetlinMachine
from time import time



#TM Hyper paremeters            Include defeault values in NoisyXOR
TM_args = {
    'number_of_clauses' : 2,           #Default    10
    'T' : 1,                          #Default   100
    's' : 1,                         #Default   1.0
    'number_of_state_bits' : 32,        #Default     2
    'depth' : 2,                        #Default     8
    'message_size' : 16,              #Default   256
    'message_bits' : 2,                 #Default     2
    'max_included_literals' : 2,       #Default     4
    'double_hashing': False             #Default False
}

EPOCHS = 1


tm = MultiClassGraphTsetlinMachine(
    **TM_args,
    grid = (1, 1, 1),
    block = (256, 1, 1)
)
values, counts = np.unique_counts(Y_train)
print(f'Class distribution: Class {values[0]}: {counts[0]}, Class {values[1]}: {counts[1]}')
print(f'Dummy accuracy = {max(counts)/sum(counts)}')

assert len(graphs_train.number_of_graph_nodes) == len(Y_train)
#Train and Eval

start_training = time()
tm.fit(graphs_train, Y_train, epochs=EPOCHS, incremental=True)
stop_training = time()

start_testing = time()
result_test = 100*(tm.predict(graphs_test) == Y_test).mean()
stop_testing = time()

result_train = 100*(tm.predict(graphs_train) == Y_train).mean()

print("Epoch: %d, Accuracy_train: %.2f,  Accuracy_test: %.2f, Training time: %.2f, Testing time: %.2f" % (3, result_train, result_test, stop_training-start_training, stop_testing-start_testing))

#log_if_best_result(result_test, train_accuracy=result_train, EPOCHS=EPOCHS, hypervector_size=HYPERVECTOR_SIZE, hypervector_bits=HYPERVECTOR_BITS, train_time=stop_training-start_training,TM_args=TM_args)

weights = tm.get_state()[1].reshape(2, -1)
for i in range(tm.number_of_clauses):
        print("Clause #%d W:(%d %d)" % (i, weights[0,i], weights[1,i]), end=' ')
        l = []
        for k in range(HYPERVECTOR_SIZE * 2):
            if tm.ta_action(0, i, k):
                if k < HYPERVECTOR_SIZE:
                    l.append("x%d" % (k))
                else:
                    l.append("NOT x%d" % (k - HYPERVECTOR_SIZE))

        # for k in range(args.message_size * 2):
        #     if tm.ta_action(1, i, k):
        #         if k < args.message_size:
        #             l.append("c%d" % (k))
        #         else:
        #             l.append("NOT c%d" % (k - args.message_size))

        print(" AND ".join(l))

print(graphs_train.hypervectors)
print(tm.hypervectors)
print(graphs_train.edge_type_id)

Initialization of sparse structure.
Class distribution: Class 1: 4003, Class 2: 3997
Dummy accuracy = 0.500375
Epoch: 3, Accuracy_train: 49.96,  Accuracy_test: 50.15, Training time: 99.54, Testing time: 0.58
Clause #0 W:(-1 -3) x52 AND x69 AND x77
Clause #1 W:(-1 -9) 
[[ 85  92]
 [ 76  11]
 [ 45 108]
 [ 20   4]
 [ 11  52]
 [105  52]
 [ 12  50]
 [ 84  56]
 [ 86  34]
 [  7  20]
 [ 31  71]
 [ 72  22]
 [ 83  36]
 [ 69  77]
 [ 61  57]
 [ 57  15]
 [ 86  82]
 [108  37]
 [ 85  41]
 [ 65  45]
 [ 91  90]
 [  8 118]
 [  1  70]]
[[ 1  3]
 [ 0 15]]
{'Right': 0, 'UpRight': 1, 'DownRight': 2, 'UpLeft': 3, 'Left': 4, 'DownLeft': 5}


## Load y for test_set and setup Graph Tsetlin machine

In [193]:
from GraphTsetlinMachine.tm import MultiClassGraphTsetlinMachine
from time import time



#TM Hyper paremeters            Include defeault values in NoisyXOR
TM_args = {
    'number_of_clauses' : 600,           #Default    10
    'T' : 300,                          #Default   100
    's' : 2.0,                          #Default   1.0
    'number_of_state_bits' : 32,        #Default     2
    'depth' : 1,                        #Default     8
    'message_size' : 256,              #Default   256
    'message_bits' : 2,                 #Default     2
    'max_included_literals' : 100,       #Default     4
    'double_hashing': True              #Default False
}

epochs_elapsed, train_time_elapsed = 0, 0

train_accuracy_across_epochs, test_accuarcy_across_epochs = [], []


tm = MultiClassGraphTsetlinMachine(
    **TM_args,
    grid = (1, 1, 1),
    block = (256, 1, 1)
)
values, counts = np.unique_counts(Y_train)
print(f'Class distribution: Class {values[0]}: {counts[0]}, Class {values[1]}: {counts[1]}')
print(f'Dummy accuracy = {max(counts)/sum(counts)}')

Initialization of sparse structure.
Class distribution: Class 1: 4003, Class 2: 3997
Dummy accuracy = 0.500375


In [194]:

assert len(graphs_train.number_of_graph_nodes) == len(Y_train)
#Train and Eval
EPOCHS = 200

#print(f'Prior to training: Accuracy_train: {100*(tm.predict(graphs_train) == Y_train).mean()},  Accuracy_test: {100*(tm.predict(graphs_test) == Y_test).mean()}')

for i in range(EPOCHS):
    epochs_elapsed += 1
    start_training = time()
    tm.fit(graphs_train, Y_train, epochs=1, incremental=True)
    stop_training = time()
    epoch_train_time = stop_training-start_training
    train_time_elapsed += epoch_train_time      


    start_testing = time()
    result_test = 100*(tm.predict(graphs_test) == Y_test).mean()
    stop_testing = time()

    result_train = 100*(tm.predict(graphs_train) == Y_train).mean()

    train_accuracy_across_epochs.append(result_train)
    test_accuarcy_across_epochs.append(result_test)

    print("Epoch: %d, Accuracy_train: %.2f,  Accuracy_test: %.2f, Accumulated Training Time: %.2f, Testing time: %.2f" % (epochs_elapsed, result_train, result_test, train_time_elapsed, stop_testing-start_testing))

    log_if_best_result(result_test, 
                       train_accuracy=result_train, 
                       EPOCHS=epochs_elapsed, 
                       hypervector_size=HYPERVECTOR_SIZE, 
                       hypervector_bits=HYPERVECTOR_BITS, 
                       train_time=train_time_elapsed,
                       TM_args=TM_args,
                       train_accuracy_across_epochs=train_accuracy_across_epochs,
                       test_accuarcy_across_epochs=test_accuarcy_across_epochs)

    if result_test == 100:
        print('100% Accuracy achieved on testset!')
        break
         



Epoch: 1, Accuracy_train: 53.49,  Accuracy_test: 53.25, Accumulated Training Time: 3.36, Testing time: 0.27
Epoch: 2, Accuracy_train: 55.33,  Accuracy_test: 54.65, Accumulated Training Time: 6.42, Testing time: 0.28
Epoch: 3, Accuracy_train: 65.56,  Accuracy_test: 63.80, Accumulated Training Time: -86.64, Testing time: 0.29
Epoch: 4, Accuracy_train: 65.64,  Accuracy_test: 64.85, Accumulated Training Time: -83.65, Testing time: 0.28
Epoch: 5, Accuracy_train: 66.67,  Accuracy_test: 65.55, Accumulated Training Time: -80.70, Testing time: 0.28
Epoch: 6, Accuracy_train: 71.59,  Accuracy_test: 70.70, Accumulated Training Time: -77.71, Testing time: 0.28
Epoch: 7, Accuracy_train: 72.81,  Accuracy_test: 71.75, Accumulated Training Time: -170.84, Testing time: 0.28
Epoch: 8, Accuracy_train: 68.55,  Accuracy_test: 67.80, Accumulated Training Time: -168.00, Testing time: 0.32
Epoch: 9, Accuracy_train: 76.30,  Accuracy_test: 76.30, Accumulated Training Time: -165.12, Testing time: 0.28
Epoch: 10, 

# 2x2 Board 100 % Accuracy

In [195]:
weights = tm.get_state()[1].reshape(2, -1)
clauses = []
for i in range(tm.number_of_clauses):
        #print("Clause #%d W:(%d %d)" % (i, weights[0,i], weights[1,i]), end=' ')
        l = []
        for k in range(HYPERVECTOR_SIZE * 2):
            if tm.ta_action(0, i, k):
                if k < HYPERVECTOR_SIZE:
                    l.append("x%d" % (k))
                else:
                    l.append("NOT x%d" % (k - HYPERVECTOR_SIZE))
        clauses.append(l)

        
        #print(" AND ".join(l))

print(f'Board Size: {board_size}, Average Clause Length: {np.mean([len(clause) for clause in clauses])}')

print(graphs_train.hypervectors)
print(tm.hypervectors)
print(graphs_train.edge_type_id)

Board Size: 7, Average Clause Length: 99.25166666666667
[[ 85  92]
 [ 76  11]
 [ 45 108]
 [ 20   4]
 [ 11  52]
 [105  52]
 [ 12  50]
 [ 84  56]
 [ 86  34]
 [  7  20]
 [ 31  71]
 [ 72  22]
 [ 83  36]
 [ 69  77]
 [ 61  57]
 [ 57  15]
 [ 86  82]
 [108  37]
 [ 85  41]
 [ 65  45]
 [ 91  90]
 [  8 118]
 [  1  70]]
[[  0 507]
 [  1 506]
 [  2 505]
 ...
 [ 85 412]
 [ 86 411]
 [ 87 410]]
{'Right': 0, 'UpRight': 1, 'DownRight': 2, 'UpLeft': 3, 'Left': 4, 'DownLeft': 5}


In [196]:
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori
import pandas as pd
import numpy as np

# Interpretability Analysis Function
def interpretability_analysis(tm, all_node_names, board_size):
    # Get clauses and state weights
    weights = tm.get_state()[1].reshape(2, -1)
    clauses = []
    for i in range(tm.number_of_clauses):
        clause_literals = []
        for k in range(HYPERVECTOR_SIZE * 2):  # Adjust to match your graph hypervector size
            if tm.ta_action(0, i, k):
                if k < HYPERVECTOR_SIZE:
                    clause_literals.append(f"x{k}")
                else:
                    clause_literals.append(f"NOT x{k - HYPERVECTOR_SIZE}")
        clauses.append(clause_literals)

    # Frequent Itemset Mining
    print("\n### Frequent Itemset Mining ###")
    
    # Convert clauses into a transaction-style binary DataFrame
    all_literals = sorted({literal for clause in clauses for literal in clause})
    binary_transactions = []
    for clause in clauses:
        binary_row = {literal: (literal in clause) for literal in all_literals}
        binary_transactions.append(binary_row)
    
    transactions_df = pd.DataFrame(binary_transactions)
    
    # Ensure no NaN values are present
    transactions_df = transactions_df.fillna(False)
    
    # Apply Apriori
    frequent_itemsets = apriori(transactions_df, min_support=0.1, use_colnames=True)
    print(frequent_itemsets)
    

    # Feature Importance
    print("\n### Feature Importance ###")
    feature_count = {}
    for clause in clauses:
        for literal in clause:
            feature_count[literal] = feature_count.get(literal, 0) + 1
    feature_importance = sorted(feature_count.items(), key=lambda x: x[1], reverse=True)
    for literal, count in feature_importance:
        print(f"{literal}: {count}")

    # Importance Matrix
    print("\n### Importance Matrix ###")
    importance_matrix = np.zeros((board_size, board_size))
    for clause in clauses:
        for literal in clause:
            if literal.startswith("x"):  # Adjust to your literal naming convention
                idx = int(literal[1:])
                row, col = divmod(idx, board_size)
                importance_matrix[row, col] += 1

    sns.heatmap(importance_matrix, annot=True, fmt=".2f")
    plt.title("Feature Importance Heatmap")
    plt.show()

    # Clause Complexity
    print("\n### Clause Complexity ###")
    for idx, clause in enumerate(clauses):
        complexity = len(clause)
        print(f"Clause #{idx}: Complexity = {complexity}")
    
    return importance_matrix


# Run Interpretability Analysis after training
#importance_matrix = interpretability_analysis(tm, all_node_names, board_size)

In [197]:
import matplotlib.pyplot as plt
import numpy as np

def plot_hex_board_importance(importance_matrix, board_size):
    """
    Plots a hexagonal board with cell importance visualized as colors.
    """
    # Create a hexagonal grid
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.set_aspect('equal')
    
    # Normalize importance values for color mapping
    normalized_importance = importance_matrix / np.max(importance_matrix)
    
    # Plot hexagons for each cell
    for row in range(board_size):
        for col in range(board_size):
            # Shift alternate rows for hex grid alignment
            x = col + (0.5 * (row % 2))
            y = -row
            hex_color = plt.cm.viridis(normalized_importance[row, col])  # Use a colormap for importance
            hexagon = plt.Polygon(
                [
                    (x + np.cos(angle), y + np.sin(angle))
                    for angle in np.linspace(0, 2 * np.pi, 7)
                ],
                edgecolor="black",
                facecolor=hex_color,
            )
            ax.add_patch(hexagon)
            # Annotate the importance value
            ax.text(x, y, f"{importance_matrix[row, col]:.2f}", ha="center", va="center", fontsize=8)

    # Set plot limits and labels
    ax.set_xlim(-1, board_size + 1)
    ax.set_ylim(-board_size, 1)
    ax.axis("off")
    plt.title("Hex Board Cell Importance")
    plt.show()

# Generate importance matrix for testing (replace with actual importance matrix)
# This should come from the `interpretability_analysis` function
#importance_matrix = np.random.rand(board_size, board_size)  # Replace with your actual matrix

# Call the function to plot the board
#plot_hex_board_importance(importance_matrix, board_size)